In [3]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3,suppress=True )

## Pedotransfer functions (PTF) 

In [4]:


class Pedotransfer:
    def __init__(self) -> None:
        pass
        # self.theta_R: float = 0.01
        # self._t: float = 1.0

    def calc_mbar(self, x):
        return 1000.0 * x

    def equation_alpha(self, Silt, Clay, OC, BD, t):
        return np.exp(
            -14.96
            + 0.03135 * Silt
            + 0.0351 * Clay
            + 0.646 * OC * (BD * 1.72)
            + 15.29 * OC
            - 0.192 * BD
            - 4.671 * BD**2
            - 0.000781 * Clay**2
            - 0.00687 * (OC * 1.72) ** 2
            + 0.0449 * (OC * 1.72) ** -1
            + 0.0663 * np.log(BD)
            + 0.1482 * np.log(OC * 1.72)
            - 0.04546 * OC * BD
            - 0.4852 * OC * (BD * 1.72)
            + 0.00673 * BD * t
        )

    def equation_theta_s(self, Silt, Clay, OC, BD, t):
        return (
            0.7919
            + 0.001691 * Clay
            - 0.29619 * BD
            - 0.000001491 * Silt**2
            + 0.0000821 * (OC * 1.72) ** 2
            + 0.02427 * Clay**-1
            + 0.01113 * Silt**-1
            + 0.01472 * np.log(Silt)
            - 0.0000733 * (OC * 1.72) * Clay
            - 0.000619 * BD * Clay
            - 0.001183 * BD * (OC * 1.72)
            - 0.0001664 * Silt * t
        )

    def equation_n(self, Silt, Clay, OC, BD, t):
        return (
            np.exp(
                -25.23
                - 0.02195 * Clay
                + 0.0074 * Silt
                - 0.194 * (OC * 1.72)
                + 45.5 * OC
                - 7.24 * BD**2
                + 0.0003658 * Clay**2
                + 0.002885 * (OC * 1.72) ** 2
                - 12.81 * BD**-1
                - 0.1524 * Silt**-1
                - 0.01958 * (OC * 1.72) ** -1
                - 0.2876 * np.log(Silt)
                - 0.0709 * np.log(OC * 1.72)
                - 44.6 * np.log(BD)
                - 0.02264 * BD * Clay
                + 0.0896 * BD * (OC * 1.72)
                + 0.00718 * Clay * t
            )
            + 1
        )

    def equation_wc(self, theta_R, theta_s, alpha, n, mbar):
        return theta_R + (theta_s - theta_R) / (
            (1 + (alpha + mbar) ** n) ** (1 - (1 / n))
        )

    def calc_WCi(self, Silt, Clay, OC, BD, theta_R, t, bar):
        theta_s = self.equation_theta_s(Silt, Clay, OC, BD, t)
        alpha = self.equation_alpha(Silt, Clay, OC, BD, t)
        n = self.equation_n(Silt, Clay, OC, BD, t)
        mbar = self.calc_mbar(bar)
        return self.equation_wc(theta_R, theta_s, alpha, n, mbar)

    def calc_M_i(self, WC_i, WC_fc, depth):
        # To convert from water content to soil moisture deficit (mm) used by RothC the following equation is used
        return (WC_i - WC_fc) * 10 * depth

    def calc_Mis(
        self,
        silt,
        clay,
        BD,
        C,
        t: float = 1.0,
        theta_R: float = 0.01,
        soil_thickness: float = 23,
    ):
        # From Annex A Pedotransfer functions
        # used to calculate the hydraulic properties
        # At Farina et al 2013
        Silt = silt
        Clay = clay
        OC = np.sum(C[:-1])
        # Water content at field capacity
        WCfc = self.calc_WCi(Silt, Clay, OC, BD, theta_R, t, -0.05)
        WCb = self.calc_WCi(Silt, Clay, OC, BD, theta_R, t, -1)
        # pwp - permanent wilting point
        WCpwp = self.calc_WCi(Silt, Clay, OC, BD, theta_R, t, -15)
        # capillary water retained at − 1000 bar
        WCc = self.calc_WCi(Silt, Clay, OC, BD, theta_R, t, -1000)
        Mb = self.calc_M_i(WCb, WCfc, soil_thickness)
        Mpwp = self.calc_M_i(WCpwp, WCfc, soil_thickness)
        Mc = self.calc_M_i(WCc, WCfc, soil_thickness)
        return Mb, Mpwp, Mc



In [15]:
SOC=69.7       #Soil organic carbon in Mg/ha 
clay=48        #Percent clay

RPMptf=(0.184*SOC + 0.1555)*(clay + 1.275)**(-0.1158)
HUMptf=(0.7148*SOC + 0.5069)*(clay + 0.3421)**(0.0184)
BIOptf=(0.014*SOC + 0.0075)*(clay + 8.8473)**(0.0567)
IOM=0.049*SOC**(1.139) # Falloon et al. (1998)
DPMptf=SOC-IOM-RPMptf-HUMptf-BIOptf

C0=np.array([DPMptf, RPMptf, BIOptf, HUMptf, IOM])

# C0=np.array([0 , 0 ,0 ,0, IOM])
print(C0)
pdf = Pedotransfer()
result = pdf.calc_Mis(silt=20,
        clay=clay,
        BD=1.3,
        C=C0 ,
        t = 1,
        theta_R = 0.01,
        soil_thickness = 23)
print(result)

[-0.014  8.266  1.236 54.051  6.161]
(0.0, 0.0, 0.0)


/var/folders/bv/qr4369nx78g7fkr6lx4h5g6r0000gn/T/ipykernel_2795/1186509868.py:11: RuntimeWarning: overflow encountered in exp
  return np.exp(
/var/folders/bv/qr4369nx78g7fkr6lx4h5g6r0000gn/T/ipykernel_2795/1186509868.py:47: RuntimeWarning: overflow encountered in exp
  np.exp(


## Rosseta